In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn
from chinese_calendar import is_workday

In [2]:
# 先製造日期在計算工作日，接著在和原本資料合併

In [3]:
def work_day(data,year):
    check_date = pd.read_csv(f"./週期資料.csv",encoding='utf-8')
    answer =  pd.DataFrame()
    date =  pd.DataFrame()
    date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
    date['週'] = date['日期'].dt.isocalendar().week
    date['年'] = date['日期'].dt.isocalendar().year
    date = date.drop(date[date['週'] == 53].index).reset_index()
    del_data =  date[date['週'] == 52].tail(1).index
    date = date.drop(date[del_data.values[0]+1:].index).reset_index()
    date["日期"] = pd.to_datetime(date["日期"] ,format='%Y/%m/%d')
    check_date["date"] = pd.to_datetime(check_date["date"] ,format='%Y/%m/%d')
    # 如果是工作日就存成true，否則存成false
    date['休假日']=date['日期'].map(lambda x:(check_date['date']==x).any())
    print(date)

    for num in range(1, 53,1):
        temp = date[date['週'] == num]
#         計算該周工作日
        holiday = len(temp[temp['休假日'] == True])
        work_day = len(temp[temp['休假日'] == False])
        date.loc[date['週']==num,'工作天數'] = work_day
        date.loc[date['週']==num,'休假天數'] = holiday
        if year == 2021 and num == 52:   
            date.loc[date['週']==num,'休假天數'] = 2

#     同一週期工作日、休假日都一樣所以只保留一筆
    date.drop_duplicates(subset='週', keep='last', inplace=True)
    date = date.drop(['日期', '休假日','level_0','index'], axis=1)
  
#     answer = pd.merge(data, date, on='週',how='outer')

    answer = pd.merge(data, date, on='週')
    answer = answer.sort_values(['帳務日期'], ascending=True).reset_index(drop=True)

#     刪除日期和工作日

    
    return answer

讀取要匯出的料號

In [12]:
# 病房
room = '加護病房'
check_no = pd.read_csv(f"../使用量、頻率排行/病房2022使用量用頻率排序/2022_{room}.csv",encoding='utf-8')
check_no = check_no[check_no['紀錄週數'] >= 13]
check_no = check_no['料號']
check_no

Series([], Name: 料號, dtype: object)

讀取資料+整理工作日資料

In [5]:

for part_no in check_no:
    print(part_no)


    # 年分
    years = [2019,2020,2021,2022]

    total_data =  pd.DataFrame()

    for year in years:
        data = pd.read_csv(f"../使用量計算/全部資料合併/{year}_{room}_合併檔.csv",encoding='utf_8_sig')
        data = data[data.columns.drop(list(data.filter(regex='Unnamed')))].reset_index(drop=True)
        data = data[data['料號'] == part_no]
        data = data.drop(data[data['數量'] < 0].index).reset_index()
        data = data.sort_values(['週'], ascending=True).reset_index(drop=True)
        # 先刪除大於52周的資料
        del_data =  data[data['週'] == 52].tail(1).index
    #     有可能沒有52週的資料
        if del_data.empty == False:
            data = data.drop(data[del_data.values[0]+1:].index).reset_index()

        # 計算工作日
        data = work_day(data,year)
        print("檢查",data)
        total_data = pd.concat([total_data, data])
        #     刪除多的欄位
    total_data = total_data[total_data.columns.drop(list(total_data.filter(regex='index')))].reset_index(drop=True)
    total_data = total_data[total_data.columns.drop(list(total_data.filter(regex='level_0')))].reset_index(drop=True)
    # 把相同的值補上
    temp = data.head(1)
    total_data['庫房'] = temp['庫房'].values[0]
    total_data['料號'] = temp['料號'].values[0]
    total_data['品名'] = temp['品名'].values[0]
    total_data['數量'].fillna(0,inplace=True)
    total_data = total_data.sort_values(['年','週'], ascending=True).reset_index(drop=True)
    # total_data.to_csv(f"./有加工作日/{room}/{part_no}.csv",encoding='utf_8_sig')
    # total_data.to_csv(f"../一個月預測一次/資料集/{room}/{part_no}.csv",encoding='utf_8_sig')
    # total_data.to_csv(f"../差值/2週資料集/{room}/{part_no}.csv",encoding='utf_8_sig')
    total_data.to_csv(f"./有加工作日/差值測試/{room}/{part_no}.csv",encoding='utf_8_sig')
    

A04030052
     level_0  index         日期   週     年    休假日
0          0      0 2019-01-01   1  2019   True
1          1      1 2019-01-02   1  2019  False
2          2      2 2019-01-03   1  2019  False
3          3      3 2019-01-04   1  2019  False
4          4      4 2019-01-05   1  2019   True
..       ...    ...        ...  ..   ...    ...
358      358    358 2019-12-25  52  2019  False
359      359    359 2019-12-26  52  2019  False
360      360    360 2019-12-27  52  2019  False
361      361    361 2019-12-28  52  2019   True
362      362    362 2019-12-29  52  2019   True

[363 rows x 6 columns]
檢查       level_0  index    庫房         料號           品名   數量        帳務日期  週    工作日  \
0           0    569  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2019-01-01  1  False   
1          18    570  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2019-01-02  1   True   
2          17    571  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2019-01-02  1   True   
3          16    572  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2019-0

C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


     level_0  index         日期   週     年    休假日
0          0      0 2020-01-01   1  2020   True
1          1      1 2020-01-02   1  2020  False
2          2      2 2020-01-03   1  2020  False
3          3      3 2020-01-04   1  2020   True
4          4      4 2020-01-05   1  2020   True
..       ...    ...        ...  ..   ...    ...
357      357    357 2020-12-23  52  2020  False
358      358    358 2020-12-24  52  2020  False
359      359    359 2020-12-25  52  2020  False
360      360    360 2020-12-26  52  2020   True
361      361    361 2020-12-27  52  2020   True

[362 rows x 6 columns]
檢查       level_0  index    庫房         料號           品名   數量        帳務日期   週  \
0           0    738  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2020-01-01   1   
1          19    739  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2020-01-01   1   
2          18    740  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2020-01-01   1   
3          17    741  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2020-01-01   1   
4          16    742  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


     level_0  index         日期   週     年    休假日
0          0      0 2022-01-01  52  2021   True
1          1      1 2022-01-02  52  2021   True
2          2      2 2022-01-03   1  2022  False
3          3      3 2022-01-04   1  2022  False
4          4      4 2022-01-05   1  2022  False
..       ...    ...        ...  ..   ...    ...
360      360    360 2022-12-27  52  2022  False
361      361    361 2022-12-28  52  2022  False
362      362    362 2022-12-29  52  2022  False
363      363    363 2022-12-30  52  2022  False
364      364    364 2022-12-31  52  2022   True

[365 rows x 6 columns]
檢查      level_0  index    庫房         料號           品名   數量        帳務日期      日期  \
0        648   5570  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2022-01-01  202201   
1        655   5572  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2022-01-01  202201   
2        656   5571  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2022-01-01  202201   
3        650   5577  DLRI  A04030052  餵食引導器 (未滅菌)  1.0  2022-01-02  202201   
4      

...

     level_0  index         日期   週     年    休假日
0          0      0 2022-01-01  52  2021   True
1          1      1 2022-01-02  52  2021   True
2          2      2 2022-01-03   1  2022  False
3          3      3 2022-01-04   1  2022  False
4          4      4 2022-01-05   1  2022  False
..       ...    ...        ...  ..   ...    ...
360      360    360 2022-12-27  52  2022  False
361      361    361 2022-12-28  52  2022  False
362      362    362 2022-12-29  52  2022  False
363      363    363 2022-12-30  52  2022  False
364      364    364 2022-12-31  52  2022   True

[365 rows x 6 columns]
檢查     index    庫房         料號               品名   數量        帳務日期      日期    計價  \
0    9309  DLRI  C09113011  加長型氣切套管  No.7.0  1.0  2022-01-19  202201  C-計價   
1    9310  DLRI  C09113011  加長型氣切套管  No.7.0  1.0  2022-01-24  202201  C-計價   
2    9311  DLRI  C09113011  加長型氣切套管  No.7.0  1.0  2022-02-04  202202  C-計價   
3    9312  DLRI  C09113011  加長型氣切套管  No.7.0  1.0  2022-02-21  202202  C-計價   
4    93

C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


     level_0  index         日期   週     年    休假日
0          0      0 2020-01-01   1  2020   True
1          1      1 2020-01-02   1  2020  False
2          2      2 2020-01-03   1  2020  False
3          3      3 2020-01-04   1  2020   True
4          4      4 2020-01-05   1  2020   True
..       ...    ...        ...  ..   ...    ...
357      357    357 2020-12-23  52  2020  False
358      358    358 2020-12-24  52  2020  False
359      359    359 2020-12-25  52  2020  False
360      360    360 2020-12-26  52  2020   True
361      361    361 2020-12-27  52  2020   True

[362 rows x 6 columns]
檢查    index    庫房         料號    品名   數量        帳務日期   週   工作日     年  工作天數  休假天數
0   7933  DLRI  B00304056  彈性紗捲  1.0  2020-07-09  28  True  2020   5.0   2.0
     level_0  index         日期   週     年    休假日
0          0      3 2021-01-04   1  2021  False
1          1      4 2021-01-05   1  2021  False
2          2      5 2021-01-06   1  2021  False
3          3      6 2021-01-07   1  2021  False
4  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\IDSL\AppData\Local\Temp\ipykernel_29716\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


     level_0  index         日期   週     年    休假日
0          0      0 2022-01-01  52  2021   True
1          1      1 2022-01-02  52  2021   True
2          2      2 2022-01-03   1  2022  False
3          3      3 2022-01-04   1  2022  False
4          4      4 2022-01-05   1  2022  False
..       ...    ...        ...  ..   ...    ...
360      360    360 2022-12-27  52  2022  False
361      361    361 2022-12-28  52  2022  False
362      362    362 2022-12-29  52  2022  False
363      363    363 2022-12-30  52  2022  False
364      364    364 2022-12-31  52  2022   True

[365 rows x 6 columns]
檢查     index    庫房         料號    品名   數量        帳務日期      日期     計價    工作日   週  \
0    8157  DLRI  B00304056  彈性紗捲  1.0  2022-02-13  202202  T-不計價  False   6   
1    8158  DLRI  B00304056  彈性紗捲  1.0  2022-03-05  202203  T-不計價  False   9   
2    8159  DLRI  B00304056  彈性紗捲  1.0  2022-03-15  202203  T-不計價   True  11   
3    8160  DLRI  B00304056  彈性紗捲  1.0  2022-03-19  202203  T-不計價  False  11   
4 

In [6]:
#抓取日歷國定假日
import requests

page = 0
isHoliday = pd.DataFrame()
while True:
    url = f"https://data.ntpc.gov.tw/api/datasets/308DCD75-6434-45BC-A95F-584DA4FED251/json?page={page}&size=1000"
    res = requests.get(url)
    resJson = res.json()
    resJson2 = pd.DataFrame(resJson)
    isHoliday = pd.concat([isHoliday, resJson2])
    page+=1
    if resJson[-1]['date']>'2023':
        break

In [7]:
isHoliday

,date,chinese,isholiday,holidaycategory,description
0,2013/1/1,...,是,放假之紀念日及節日,全國各機關學校放假一日。
1,2013/1/5,,是,星期六、星期日,
2,2013/1/6,,是,星期六、星期日,
3,2013/1/12,,是,星期六、星期日,
4,2013/1/13,,是,星期六、星期日,
...,...,...,...,...,...
310,2023/12/17,,是,星期六、星期日,
311,2023/12/23,,是,星期六、星期日,
312,2023/12/24,,是,星期六、星期日,
313,2023/12/30,,是,星期六、星期日,


In [8]:
isHoliday
isHoliday['date'] = pd.to_datetime(isHoliday['date'])
isHoliday['isholiday']=isHoliday.apply(lambda row: 1 if row['isholiday']=='是' else 0, axis=1)
#1為國定假日0則不是
isHoliday = isHoliday[['date','isholiday']]
# isHoliday['Date'] = pd.to_datetime(isHoliday['Date'])
isHoliday = isHoliday[(isHoliday['date']<'2023')&(isHoliday['date']>='2019')]

isHoliday['週'] = isHoliday['date'].dt.isocalendar().week
# isHoliday['年'] = isHoliday['date'].dt.isocalendar().year
# isHoliday['月'] = isHoliday['date'].dt.month
# isHoliday[(isHoliday['年'] == 2021)&(isHoliday['月'] == 2)]
# isHoliday.to_csv(f"./週期資料.csv")
isHoliday

,date,isholiday,週
718,2019-01-01,1,1
719,2019-01-05,1,1
720,2019-01-06,1,1
721,2019-01-12,1,2
722,2019-01-13,1,2
...,...,...,...
188,2022-12-17,1,50
189,2022-12-18,1,50
190,2022-12-24,1,51
191,2022-12-25,1,51


In [9]:
total_data.tail(1500)

,庫房,料號,品名,數量,帳務日期,工作日,週,年,工作天數,休假天數,日期,計價
0,DLRI,B00304056,彈性紗捲,1.0,2020-07-09,True,28,2020,5.0,2.0,NaN,NaN
1,DLRI,B00304056,彈性紗捲,1.0,2021-01-20,True,3,2021,5.0,2.0,NaN,NaN
2,DLRI,B00304056,彈性紗捲,2.0,2021-03-19,True,11,2021,5.0,2.0,NaN,NaN
3,DLRI,B00304056,彈性紗捲,2.0,2021-03-20,False,11,2021,5.0,2.0,NaN,NaN
4,DLRI,B00304056,彈性紗捲,1.0,2021-03-21,False,11,2021,5.0,2.0,NaN,NaN
5,DLRI,B00304056,彈性紗捲,1.0,2021-03-24,True,12,2021,5.0,2.0,NaN,NaN
6,DLRI,B00304056,彈性紗捲,2.0,2021-03-26,True,12,2021,5.0,2.0,NaN,NaN
7,DLRI,B00304056,彈性紗捲,2.0,2021-03-27,False,12,2021,5.0,2.0,NaN,NaN
8,DLRI,B00304056,彈性紗捲,1.0,2021-03-28,False,12,2021,5.0,2.0,NaN,NaN
9,DLRI,B00304056,彈性紗捲,1.0,2021-03-29,True,13,2021,4.0,3.0,NaN,NaN
